In [4]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Backtest, Strategy

In [5]:
df = pd.read_csv('data/spgi-60d.csv')
df['Datetime'] = pd.to_datetime(df['Datetime'], utc=True)
df.set_index('Datetime', inplace=True)

In [6]:
df['EMA_50'] = ta.ema(df['Close'], length=50)
df['Sentiment'] = np.where(df['Close'] > df['EMA_50'], 'B', np.where(df['Close'] < df['EMA_50'], 'S', 'N'))
df.ta.macd(fast=12, slow=26, signal=9, append=True)
df['Signal'] = np.where(df['MACDh_12_26_9'] > 0, 'B', np.where(df['MACDh_12_26_9'] < 0, 'S', 'N'))
df['ATR'] = ta.atr(df.High, df.Low, df.Close, length=14)
df = df.dropna(subset=['MACDh_12_26_9', 'ATR', 'EMA_50'])
df.to_csv('data/spgi-60d-ema-macd.csv')
df

Open        High         Low       Close  \
Datetime                                                                    
2023-12-27 20:15:00+00:00  439.869995  440.089996  439.519989  439.809998   
2023-12-27 20:30:00+00:00  439.709991  440.029999  439.290009  439.429993   
2023-12-27 20:45:00+00:00  439.429993  440.290009  439.210999  440.000000   
2023-12-28 14:30:00+00:00  439.660004  443.470001  439.019989  440.910004   
2023-12-28 14:45:00+00:00  440.875000  442.290009  440.875000  441.934998   
...                               ...         ...         ...         ...   
2024-03-20 19:45:00+00:00  425.567505  426.059998  424.970001  426.049988   
2024-03-21 13:30:00+00:00  427.350006  427.772888  426.005005  427.380005   
2024-03-21 13:45:00+00:00  427.380005  427.600006  426.114990  427.549988   
2024-03-21 14:00:00+00:00  427.454987  428.079987  427.385010  427.755005   
2024-03-21 14:15:00+00:00  427.850006  428.679993  427.725006  428.679993   

                           Volume  Dividends  Stock Splits      EMA_50  \
Datetime                                                                 
2023-12-27 20:15:00+00:00   27712        0.0           0.0  437.332154   
2023-12-27 20:30:00+00:00   41907        0.0           0.0  437.414422   
2023-12-27 20:45:00+00:00  111968        0.0           0.0  437.515817   
2023-12-28 14:30:00+00:00   49931        0.0           0.0  437.648923   
2023-12-28 14:45:00+00:00   14085        0.0           0.0  437.817004   
...                           ...        ...           ...         ...   
2024-03-20 19:45:00+00:00  156353        0.0           0.0  423.702496   
2024-03-21 13:30:00+00:00   59331        0.0           0.0  423.846712   
2024-03-21 13:45:00+00:00   71071        0.0           0.0  423.991939   
2024-03-21 14:00:00+00:00   54948        0.0           0.0  424.139510   
2024-03-21 14:15:00+00:00   45334        0.0           0.0  424.317568   

                          Sentiment  MACD_12_26_9  MACDh_12_26_9  \
Datetime                                                           
2023-12-27 20:15:00+00:00         B      0.745482      -0.019159   
2023-12-27 20:30:00+00:00         B      0.693549      -0.056873   
2023-12-27 20:45:00+00:00         B      0.690428      -0.047995   
2023-12-28 14:30:00+00:00         B      0.752708       0.011427   
2023-12-28 14:45:00+00:00         B      0.874690       0.106728   
...                             ...           ...            ...   
2024-03-20 19:45:00+00:00         B      0.577670       0.242933   
2024-03-21 13:30:00+00:00         B      0.751835       0.333679   
2024-03-21 13:45:00+00:00         B      0.893282       0.380100   
2024-03-21 14:00:00+00:00         B      1.010276       0.397676   
2024-03-21 14:15:00+00:00         B      1.164214       0.441291   

                           MACDs_12_26_9 Signal       ATR  
Datetime                                                   
2023-12-27 20:15:00+00:00       0.764641      S  0.648425  
2023-12-27 20:30:00+00:00       0.750423      S  0.655131  
2023-12-27 20:45:00+00:00       0.738424      S  0.686115  
2023-12-28 14:30:00+00:00       0.741281      B  0.960789  
2023-12-28 14:45:00+00:00       0.767962      B  0.993885  
...                                  ...    ...       ...  
2024-03-20 19:45:00+00:00       0.334737      B  1.022640  
2024-03-21 13:30:00+00:00       0.418157      B  1.075872  
2024-03-21 13:45:00+00:00       0.513182      B  1.105097  
2024-03-21 14:00:00+00:00       0.612601      B  1.075802  
2024-03-21 14:15:00+00:00       0.722923      B  1.067173  

[1489 rows x 14 columns]

In [7]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close']), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['MACD_12_26_9'], mode='lines', name='MACD Line'), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['MACDs_12_26_9'], mode='lines', name='Signal Line'), row=2, col=1)
fig.add_trace(go.Bar(x=df.index, y=df['MACDh_12_26_9'], name='Histogram'), row=2, col=1)

fig.show()


In [17]:
class MyStrategy(Strategy):
  size = 100
  slCoefficient = 1.1
  tpCoefficient = 1.5

  def init(self):
    super().init()
    self.signal = self.data.Signal

  def next(self):
    super().next()
    # stopLossAtr = self.slCoefficient * (self.data.ATR[-1] if not np.isnan(self.data.ATR[-1]) else 6)
    stopLossAtr = self.slCoefficient * self.data.ATR[-1]

    # if self.signal == 'B':
    #   self.position.close()
    #   stopLoss = self.data.Close[-1] - stopLossAtr
    #   takeProfit = self.data.Close[-1] + self.tpCoefficient * stopLossAtr
    #   self.buy(sl=stopLoss, tp=takeProfit, size=self.size)

    # elif self.signal == 'S':
    #   self.position.close()
    #   stopLoss = self.data.Close[-1] + stopLossAtr
    #   takeProfit = self.data.Close[-1] - self.tpCoefficient * stopLossAtr
    #   self.sell(sl=stopLoss, tp=takeProfit, size=self.size)
    if self.signal == 'B' and len(self.trades) == 0:
      stopLoss = self.data.Close[-1] - stopLossAtr
      takeProfit = self.data.Close[-1] + self.tpCoefficient * stopLossAtr
      self.buy(sl=stopLoss, tp=takeProfit, size=self.size)

    elif self.signal == 'S' and len(self.trades) == 0:
      stopLoss = self.data.Close[-1] + stopLossAtr
      takeProfit = self.data.Close[-1] - self.tpCoefficient * stopLossAtr
      self.sell(sl=stopLoss, tp=takeProfit, size=self.size)


bt = Backtest(df, MyStrategy, cash=1000000000, commission=.002)

results = bt.run()
bt.plot()
# Print the backtesting results
results['_equity_curve'].to_csv('data/spig-60d-equity_curve.csv')
results['_trades'].to_csv('data/spig-60d-trades.csv')
results


/Users/andrew/.pyenv/versions/3.11.4/lib/python3.11/site-packages/backtesting/_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

/Users/andrew/.pyenv/versions/3.11.4/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

/Users/andrew/.pyenv/versions/3.11.4/lib/python3.11/site-packages/backtesting/_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

/Users/andrew/.pyenv/versions/3.11.4/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

/Users/andrew/.pyenv/versions/3.11.4/lib/python3.11/site-packages/backtesting/

Start                     2023-12-27 20:15...
End                       2024-03-21 14:15...
Duration                     84 days 18:00:00
Exposure Time [%]                   99.865682
Equity Final [$]             999977600.463441
Equity Peak [$]             1000000094.066055
Return [%]                           -0.00224
Buy & Hold Return [%]               -2.530639
Return (Ann.) [%]                   -0.009554
Volatility (Ann.) [%]                0.000686
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.002252
Avg. Drawdown [%]                   -0.000756
Max. Drawdown Duration       83 days 19:00:00
Avg. Drawdown Duration       28 days 05:35:00
# Trades                                  283
Win Rate [%]                        40.282686
Best Trade [%]                       1.447465
Worst Trade [%]                     -6.321873
Avg. Trade [%]                    